In [ ]:
import traitlets
from traitlets.config.configurable import SingletonConfigurable
import numpy as np
import enum
import cv2
import ipywidgets as widgets
from torchvision import models
fcn = models.segmentation.fcn_resnet101(pretrained=True).eval()
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as T

import sys
sys.path.append('..')
from opencv_course.video import Camera

trf = T.Compose([T.Resize(256),
                 T.CenterCrop(224),
                 T.ToTensor(), 
                 T.Normalize(mean = [0.485, 0.456, 0.406], 
                             std = [0.229, 0.224, 0.225])])

def decode_segmap(image, nc=21):
  
    label_colors = np.array([(0, 0, 0),  # 0=background
               # 1=aeroplane, 2=bicycle, 3=bird, 4=boat, 5=bottle
               (128, 0, 0), (0, 128, 0), (128, 128, 0), (0, 0, 128), (128, 0, 128),
               # 6=bus, 7=car, 8=cat, 9=chair, 10=cow
               (0, 128, 128), (128, 128, 128), (64, 0, 0), (192, 0, 0), (64, 128, 0),
               # 11=dining table, 12=dog, 13=horse, 14=motorbike, 15=person
               (192, 128, 0), (64, 0, 128), (192, 0, 128), (64, 128, 128), (255, 255, 255),
               # 16=potted plant, 17=sheep, 18=sofa, 19=train, 20=tv/monitor
               (0, 64, 0), (128, 64, 0), (0, 192, 0), (128, 192, 0), (0, 64, 128)])

    r = np.zeros_like(image).astype(np.uint8)
    g = np.zeros_like(image).astype(np.uint8)
    b = np.zeros_like(image).astype(np.uint8)

    for l in range(0, nc):
        idx = image == l
        r[idx] = label_colors[l, 0]
        g[idx] = label_colors[l, 1]
        b[idx] = label_colors[l, 2]

    bgr = np.stack([b, g, r], axis=2)
    return bgr

    
def bgr8_to_jpeg(value, quality=10):
    global trf
    img = Image.fromarray(value.copy())
    inp = trf(img).unsqueeze(0)
    out = fcn(inp)['out']
    om = torch.argmax(out.squeeze(), dim=0).detach().cpu().numpy()
    image = decode_segmap(om)
    (h, w, r) = value.shape
    resized = Image.fromarray(value).resize((256,256), Image.ANTIALIAS)
    corped = cv2.subtract(np.array(resized), 255-image)
    return bytes(cv2.imencode('.jpg', corped)[1])

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

camera = Camera(width=500, height=500, fps=10)

image = widgets.Image(format='jpeg', width=500, height=500)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

In [ ]:
camera.unobserve_all()
camera.stop()